In [1]:
from carUtils import *

In [3]:
from linesUtils import *

In [ ]:
class Enviroment:
    def __init__(self,car,line_generator=None):
        self.car=car
        self.line_generator=line_generator
        
    def reset(self):
        if self.line_generator:
            self.lines,self.startX,self.startY,startOrientation,self.endX,self.endY=self.line_generator()
        else:
            self.lines=[]

            self.lines+=make_curve(90, 0.5, start_point=[0,0],n_lines=50,left=True)
            self.lines+=make_curve(-90, 0.5, start_point=[0,0],n_lines=10,left=False)
            self.startX,self.startY=(0.5,-0.5)
            self.endX,self.endY=(-0.5,0.5)
            startOrientation=
        